In [1]:
from nba_api.stats.endpoints import boxscoresummaryv2,leaguedashteamstats, leaguestandingsv3, teamgamelog
import pandas as pd
import numpy as np
import json, csv, time

In [2]:
teams = pd.read_csv("teams.csv")
players = pd.read_csv("player_stats_2024.csv")
players.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,AST,STL,BLK,TOV,PF,PTS,PPG,APG,PER,FULL_NAME
0,1630173,2024-25,0,1610612752,NYK,25.0,57,10,1170.0,164,...,55,47,42,45,81,379,6.6,1.0,16.2,Precious Achiuwa
1,203500,2024-25,0,1610612745,HOU,31.0,58,3,794.0,91,...,66,22,28,54,60,225,3.9,1.1,19.3,Steven Adams
2,1628389,2024-25,0,1610612748,MIA,27.0,78,78,2674.0,540,...,337,98,53,161,162,1410,18.1,4.3,22.9,Bam Adebayo
3,1630534,2024-25,0,1610612761,TOR,24.0,64,45,1739.0,266,...,98,58,30,54,125,667,10.4,1.5,15.3,Ochai Agbaji
4,1630583,2024-25,0,1610612763,MEM,24.0,65,16,1660.0,313,...,188,52,29,71,76,811,12.5,2.9,21.8,Santi Aldama


In [3]:
def set_injured_status(season, season_type, teams):
    teams = teams.copy()
    teams['HighValueInjured'] = 0
   
    for i, team_row in teams.iterrows():
        team_id = str(team_row['TEAM_ID'])
        inactive_players = set()
        injured_status_flag = 0

        last_3_games = teamgamelog.TeamGameLog(season=season,season_type_all_star=season_type,team_id=team_id).get_data_frames()[0].head(3)
        for game in last_3_games['Game_ID']:
            summary = boxscoresummaryv2.BoxScoreSummaryV2(game)
            inactive_df = summary.inactive_players.get_data_frame()

            for player in inactive_df['PLAYER_ID']:
                player_team = str(players.loc[players['PLAYER_ID']==player, 'TEAM_ID'].squeeze())
                if player_team==team_id:
                    inactive_players.add(player)
            time.sleep(0.6)
        for player in inactive_players:
            try:
                player_per = float(players.loc[players['PLAYER_ID']==player, 'PER'].squeeze())
                player_min = int(players.loc[players['PLAYER_ID']==player, 'MIN'].squeeze())

                if player_per > 25 and player_min > 750:
                    injured_status_flag = 1
                    break
            except (KeyError, ValueError):
                continue
        teams.loc[i,'HighValueInjured'] = injured_status_flag
    return teams


In [9]:
set_injured_status("2024-25","Regular Season",teams)

,TEAM_NAME,TEAM_ID,HighValueInjured
0,Atlanta Hawks,1610612737,0
1,Boston Celtics,1610612738,1
2,Cleveland Cavaliers,1610612739,0
3,New Orleans Pelicans,1610612740,0
4,Chicago Bulls,1610612741,0
5,Dallas Mavericks,1610612742,0
6,Denver Nuggets,1610612743,0
7,Golden State Warriors,1610612744,0
8,Houston Rockets,1610612745,1
9,Los Angeles Clippers,1610612746,0


In [4]:
def get_team_stats(season, season_type):

    columns = ['TEAM_ABBREVIATION', 'W', 'L', 'W_PCT', 'FGM_RANK', 'FG3M_RANK', 'PLUS_MINUS_RANK', 'REB_RANK', 'AST_RANK',
     'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'PF_RANK', 'PTS_RANK','DivisionRank', 'HOME_WINS', 'ROAD_WINS', 'CurrentHomeStreak',
     'CurrentRoadStreak', 'CurrentStreak', 'HighValueInjured']

    boxscore_stats = leaguedashteamstats.LeagueDashTeamStats(season=season, season_type_all_star=season_type, measure_type_detailed_defense='Base').get_data_frames()[0]
    league_standings = leaguestandingsv3.LeagueStandingsV3(season=season, season_type=season_type).get_data_frames()[0]

    league_standings[['HOME_WINS', 'HOME_LOSSES']] = league_standings['HOME'].str.split("-", expand=True).astype(int)
    league_standings['HOME_WIN_PCT'] = round(league_standings['HOME_WINS'] / (league_standings['HOME_WINS'] + league_standings['HOME_LOSSES']),2)

    league_standings[['ROAD_WINS', 'ROAD_LOSSES']] = league_standings['ROAD'].str.split("-", expand=True).astype(int)
    league_standings['ROAD_WIN_PCT'] = round(league_standings['ROAD_WINS'] / (league_standings['ROAD_WINS'] + league_standings['ROAD_LOSSES']),2)
    league_standings['TEAM_ID'] = league_standings['TeamID']
    league_standings = league_standings.drop(columns=['TeamID'])
    team_stats = pd.merge(boxscore_stats, league_standings, on='TEAM_ID')
    injured_standings = set_injured_status(season, season_type,teams)
    injured_standings = injured_standings.drop(columns='TEAM_NAME')
    team_stats = pd.merge(team_stats, injured_standings, on='TEAM_ID')

    return team_stats[columns]

In [5]:
team_stats = get_team_stats('2024-25', 'Regular Season')
team_stats.to_csv('master_team_stats_2024.csv', index=False)